In [17]:
import json
import numpy as np
import matplotlib.pyplot as plt



if __name__ == '__main__':
    ABC_cycles_mean = json.load(open("ABC_cycles_mean.json"))
    Orig_cycles_mean = json.load(open("Orig_cycles_mean.json"))
    Orig_cycles_sigma = json.load(open("Orig_cycles_sigma.json"))
    # Due to not all the results are available(some may be -1, which indicating failure)
    AvailableKeys = []
    for target, cycles in ABC_cycles_mean.items():
        if cycles != -1:
            AvailableKeys.append(target)
    SpeedupDict = {}
    '''
    SpeedupDict contains all the information that we need =)
    '''
    for target in AvailableKeys:
        new = ABC_cycles_mean[target]
        old = Orig_cycles_mean[target]
        speedup = (old - new) / old
        sigma = Orig_cycles_sigma[target]
        SpeedupDict[target] = {}
        SpeedupDict[target]['new'] = new
        SpeedupDict[target]['old'] = old
        SpeedupDict[target]['speedup'] = speedup
        SpeedupDict[target]['sigma'] = sigma
    '''
    Discard the speedup or slow-down less than twice sigma
    '''
    SelectedSpeedupDict = SpeedupDict.copy()
    for target, info in SpeedupDict.items():
        if abs(info['new'] - info['old']) < 2*info['sigma']:
            SelectedSpeedupDict.pop(target, None)
    print("Number of programs that are our targets: {}".format(len(Orig_cycles_mean)))
    print("Number of programs that built successfully by ABC: {}".format(len(SpeedupDict)))
    print("Number of programs that speedup by ABC(larger than twice sigma): {}".format(len(SelectedSpeedupDict)))
    print("-------------------------------------------------------------------------------------")
        
        
        


Number of programs that are our targets: 272
Number of programs that built successfully by ABC: 258
Number of programs that speedup by ABC(larger than twice sigma): 148
-------------------------------------------------------------------------------------
